In [2]:
import urllib3
urllib3.disable_warnings()
import json
import datetime
import csv
import time
import pandas as pd

In [3]:
app_id = "133254600775938"

app_secret = "c24db12ccb6a583d8e212600c1bed43e" # DO NOT SHARE WITH ANYONE!

access_token = app_id + "|" + app_secret

#page_id = 'nytimes'
#page_id = 'www.salesintacc.com.ar'
#page_id = 'upsoclsabores'
page_id = 'cucinareok'

In [7]:
def request_until_succeed(url):
    http = urllib3.PoolManager()
    req = http.request('GET', url)
    success = False
    while success is False:
        try:
            response = http.urlopen('GET',url)
            if response.status == 200:
                success = True
        except:
            time.sleep(5)
            
            print ("Error for URL %s: %s" % (url, datetime.datetime.now()))

    return req

In [8]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.11" #cambié v2.4 a v2.11, funcionaba igual con v2.4
    node = "/" + page_id + "/feed"
    parameters = "/?fields=message,link,created_time,type,name,id,comments,from,attachment,shares&limit=%s&access_token=%s" % (num_statuses, access_token)
    url = base + node + parameters
    
    # Esto está cambiado respecto al ejemplo
    http = urllib3.PoolManager()
    req = http.request('GET', url)
    data = json.loads(request_until_succeed(url).data.decode('latin-1'))
    return data
    

In [9]:
#Estoy levantando 100 posts
salida_all = getFacebookPageFeedData(page_id, access_token, 100)

columns = ['post_from', 'post_id', 'post_name', 'post_type', 'post_message', 'post_link', 'post_shares', 'created_time']

df_posts = pd.DataFrame(columns=columns)

In [10]:
def scrapPosts(post):
    post_from = '' if 'from' not in post.keys() else post['from']['id']
    post_id = post['id']
    post_name = '' if 'name' not in post.keys() else post['name']
    post_type = '' if 'type' not in post.keys() else post['type']
    post_message = '' if 'message' not in post.keys() else post['message']
    post_link = '' if 'link' not in post.keys() else post['link']
    post_shares = '' if 'shares' not in post.keys() else post['shares']['count']
    created_time = datetime.datetime.strptime(post['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    created_time = created_time + datetime.timedelta(hours=-5) # EST
    created_time = created_time.strftime('%Y-%m-%d %H:%M:%S')    

    return(post_from, post_id, post_name, post_type, post_message, post_link, post_shares, created_time)


In [11]:
n_posts = len(salida_all['data'])

for i in range(n_posts):
    df = pd.DataFrame(list(scrapPosts(salida_all["data"][i])),index = ['post_from', 'post_id', 'post_name', 'post_type', 'post_message', 'post_link', 'post_shares', 'created_time']).transpose()
    df_posts = df_posts.append(df, ignore_index=True)    


In [12]:
df_posts.to_csv("df_posts.csv")

In [13]:
def scrapComments(pos, com):

    post_id = salida_all["data"][pos]["id"]
    comm_id = salida_all["data"][pos]["comments"]["data"][com]["id"]
    comm_msg = salida_all["data"][pos]["comments"]["data"][com]["message"]
    comm_date = datetime.datetime.strptime(salida_all["data"][pos]["comments"]["data"][com]["created_time"],'%Y-%m-%dT%H:%M:%S+0000')
    comm_date = comm_date + datetime.timedelta(hours=-5) # EST
    comm_date = comm_date.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    return(post_id, comm_id, comm_msg, comm_date)    


In [14]:
df_comms = pd.DataFrame(columns=['post_id','comm_id','comm_msg','comm_date'])

In [17]:
#n_posts = len(salida_all['data'])

for i in range(n_posts):
    if len(salida_all["data"][i]) == 8:
        continue
    ncomm = len(salida_all["data"][i]["comments"]["data"])
    for c in range(ncomm):
        df = pd.DataFrame(list(scrapComments(i,c)), index=['post_id','comm_id','comm_msg','comm_date']).transpose()
        df_comms = df_comms.append(df, ignore_index=True)
        
        

KeyError: 'comments'

In [202]:
df_comms.to_csv("df_comms.csv")

In [16]:
df_comms

,post_id,comm_id,comm_msg,comm_date
0,262755230730537_544450155894375,544450155894375_544564402549617,,2017-11-18 16:13:35
1,262755230730537_544450155894375,544450155894375_544559382550119,Romy Naxd iooooo jajajaj,2017-11-18 15:56:40
2,262755230730537_544450155894375,544450155894375_544548659217858,"Me gusta , mira Cielo H. Ferrari María Luján F...",2017-11-18 15:25:52
3,262755230730537_544450155894375,544450155894375_544580189214705,Qué rico !!,2017-11-18 17:25:16
4,262755230730537_544450155894375,544450155894375_544570049215719,Feliz cumple Sofi !!,2017-11-18 16:39:28
5,262755230730537_544450155894375,544450155894375_544545085884882,Ana Allende este era el que te decia ma,2017-11-18 15:16:37
6,262755230730537_544450155894375,544450155894375_544576525881738,Uno así para mi cumple Natalia Mozo!,2017-11-18 17:10:50
7,262755230730537_544450155894375,544450155894375_544552712550786,Flá mira mor,2017-11-18 15:44:35
8,262755230730537_544450155894375,544450155894375_544581662547891,.,2017-11-18 17:33:08
9,262755230730537_544450155894375,544450155894375_544568759215848,,2017-11-18 16:34:01


In [25]:
import os
os.getcwd()


'C:\\Users\\Pamela\\Documents\\GitHub\\NLP Facebook'

In [33]:

java_path = "C:/Program Files (x86)/Java/jre1.8.0_151/bin/java.exe"
os.environ['JAVAHOME'] = java_path


# -*- coding: utf-8 -*-

from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

st = StanfordNERTagger('../../stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz',
					   '../../stanford-ner/stanford-ner.jar',
					   encoding='utf-8')

In [32]:
text = 'While in France, Christine lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.'

tokenized_text = word_tokenize(text)
classified_text = st.tag(tokenized_text)

print(classified_text)

[('While', 'O'), ('in', 'O'), ('France', 'LOCATION'), (',', 'O'), ('Christine', 'PERSON'), ('lagarde', 'PERSON'), ('discussed', 'O'), ('short-term', 'O'), ('stimulus', 'O'), ('efforts', 'O'), ('in', 'O'), ('a', 'O'), ('recent', 'O'), ('interview', 'O'), ('with', 'O'), ('the', 'O'), ('Wall', 'ORGANIZATION'), ('Street', 'ORGANIZATION'), ('Journal', 'ORGANIZATION'), ('.', 'O')]
